In [1]:
!pip install pandas docxtpl openpyxl

In [6]:
import pandas as pd
from docxtpl import DocxTemplate
from datetime import date
import os

def generar_facturas(excel_path, template_path):
    """
    Genera facturas en formato Word para cada cliente a partir de un archivo Excel.
    """
    try:
        df = pd.read_excel(excel_path)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{excel_path}'.")
        return

 
    clientes_agrupados = df.groupby('Cliente')
    
    try:
        doc = DocxTemplate(template_path)
    except FileNotFoundError:
        print(f"Error: No se encontró la plantilla de Word '{template_path}'.")
        return

    fecha_emision = date.today().strftime("%d/%m/%Y")
    vendedor_info = "Tu Nombre / Nombre de la Empresa"

    for cliente, datos_cliente in clientes_agrupados:
        
        productos_lista = []
        total_factura = 0.0

        for _, fila in datos_cliente.iterrows():
            total_producto = fila['Cantidad'] * fila['Precio Unitario']
            productos_lista.append({
                'producto': fila['Producto'],
                'cantidad': fila['Cantidad'],
                'precio_unitario': f"{fila['Precio Unitario']:.2f}",
                'total_producto': f"{total_producto:.2f}"
            })
            total_factura += total_producto
            
        context = {
            'cliente': cliente,
            'fecha': fecha_emision,
            'vendedor': vendedor_info,
            'productos': productos_lista,
            'total_factura': f"{total_factura:.2f}"
        }
        
        doc.render(context)
        output_filename = f"factura_{cliente.replace(' ', '_')}.docx"
        doc.save(output_filename)
        
        print(f"Factura generada para {cliente}: {output_filename}")


excel_file = "factura.xlsx"
word_template = "factura text.docx"  

generar_facturas(excel_file, word_template)

Factura generada para Ana Gómez: factura_Ana_Gómez.docx
Factura generada para Juan Pérez: factura_Juan_Pérez.docx
Factura generada para Luis Castro: factura_Luis_Castro.docx
